In [ ]:
import os
import numpy as np
import sklearn
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, make_scorer
from sklearn import svm
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import RobustScaler

In [2]:
def specificity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()
    return tn/(tn+fp)

In [3]:
# Read data
int_ict_data = np.load('/kaggle/input/train-test-data-bsp-project/int_ict_feats.npy')
int_ict_data = int_ict_data.reshape(int_ict_data.shape[0],10)
int_ict_labels = np.load('/kaggle/input/train-test-data-bsp-project/int_ict_label.npy')

norm_ict_data = np.load('/kaggle/input/train-test-data-bsp-project/norm_ict_feats.npy')
norm_ict_data = norm_ict_data.reshape(norm_ict_data.shape[0],10)
norm_ict_labels = np.load('/kaggle/input/train-test-data-bsp-project/norm_ict_label.npy')

In [ ]:
intict_train_data, intict_test_data, intict_train_labels, intict_test_labels = train_test_split(int_ict_data, int_ict_labels, test_size = 0.2, random_state=42)

classifier = svm.SVC(C=100, kernel='rbf', gamma=0.032)
classifier.fit(intict_train_data, np.ravel(intict_train_labels))

intict_test_preds = classifier.predict(intict_test_data)
accuracy = accuracy_score(intict_test_labels, intict_test_preds)
print(f'accuracy = {accuracy:.4f}\n')

confusion_mat = confusion_matrix(intict_test_labels, intict_test_preds)
print(f'\nconfusion matrix:\n {confusion_mat}\n')

print(classification_report(intict_test_labels, intict_test_preds))


scaler = RobustScaler()
intict_train_feats_scaled = scaler.fit_transform(intict_train_data)


sensitivity_scorer = make_scorer(recall_score, pos_label=1)
specificity_scorer = make_scorer(specificity)

scoring = {'sensitivity': sensitivity_scorer, 'specificity':specificity_scorer}

cv_results = cross_validate(classifier, intict_train_feats_scaled, np.ravel(intict_train_labels), cv=StratifiedKFold(4), scoring=scoring)


print('cross validation fold sensitivities: ',cv_results['test_sensitivity'])
print('cross validation fold specificities: ',cv_results['test_specificity'])

print('average sensitivity: ',np.mean(cv_results['test_sensitivity']))
print('average specificity: ',np.mean(cv_results['test_specificity']))


accuracy = 0.8938


confusion matrix:
 [[78  2]
 [15 65]]

              precision    recall  f1-score   support

           0       0.84      0.97      0.90        80
           1       0.97      0.81      0.88        80

    accuracy                           0.89       160
   macro avg       0.90      0.89      0.89       160
weighted avg       0.90      0.89      0.89       160

cross validation fold sensitivities:  [0.95   0.975  0.9625 0.9125]
cross validation fold specificities:  [0.95   0.9875 0.9875 0.925 ]
average sensitivity:  0.95
average specificity:  0.9624999999999999
